In [ ]:
pip install kserve

In [ ]:
from kserve import RESTConfig, InferenceRESTClient, InferRequest, InferInput
from PIL import Image
import matplotlib.pyplot as plt
from torchvision import datasets, transforms
from random import randrange
import numpy as np
import os
import json
import requests

In [ ]:
# Assuming dataset is already downloaded from the basic demo

In [ ]:
# Dataset
ds_dog_breeds_path = "./dog-breeds-dataset"
test_ds_dog_breeds_path = os.path.join(ds_dog_breeds_path, "test")
ds_dog_breeds = datasets.ImageFolder(test_ds_dog_breeds_path)
dog_breeds_classes = ds_dog_breeds.find_classes(test_ds_dog_breeds_path)[0]

print(f"Classes dog_breeds_classifier: {dog_breeds_classes}")

In [ ]:
# Show sample image
fig = plt.figure(figsize=(1, 1))
sample_idx = randrange(len(ds_dog_breeds))

img, label = ds_dog_breeds[sample_idx]
fig.add_subplot(1, 1, 1)

plt.title(dog_breeds_classes[label])
plt.axis("off")
plt.imshow(img)
plt.show()# Dataset
ds_dog_breeds_path = "./dog-breeds-dataset"
test_ds_dog_breeds_path = os.path.join(ds_dog_breeds_path, "test")
ds_dog_breeds = datasets.ImageFolder(test_ds_dog_breeds_path)
dog_breeds_classes = ds_dog_breeds.find_classes(test_ds_dog_breeds_path)[0]

print(f"Classes dog_breeds_classifier: {dog_breeds_classes}")

img_array = np.array(img.convert('RGB'))

if img_array.ndim == 3:
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
print(img_array.shape)

In [ ]:
# Inference
model_name = "dog-poem-pipeline" # The name of the graph
request = InferRequest(
    model_name=model_name,
    
    # This 'infer_inputs' block is for the first step (dog-breeds-classifier).
    infer_inputs=[
        InferInput(
            name="data",
            shape=list(img_array.shape),
            datatype="FP32",
            data=img_array.tolist()
        )
    ],
    
    # This 'parameters' block will be passed to your TopicPromptBuilder transformer.
    parameters={
        "class_labels": dog_breeds_classes,
        "initial_prompt": "Write a fun poem",
        "target_model_name": "vllm-llama3-8b"
    }
)
#    You can get this with: oc get ig dog-poem-pipeline -o jsonpath='{.status.url}'
base_url = "https://dog-poem-pipeline-modelserving-demo.apps.rosa.n1t3u2f3w1s0b1d.kkw2.p3.openshiftapps.com"

In [ ]:
cfg = RESTConfig(verify=False)
client = InferenceRESTClient(cfg)

infer_response = await client.infer(
    base_url,
    request,
    model_name=request.model_name,
    is_graph_endpoint=True
)

print("\n--- Final Response from Graph ---")
print(infer_response)